In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
import pandas as pd
import re
import tensorflow as tf
print(tf.__version__)
import nltk
nltk.download("wordnet")
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
print(pd.__version__)

2.15.0
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2.2.2


In [7]:
df=pd.read_csv("/kaggle/input/fake-news/train.csv")
print(f"total number of rows are:{len(df.index)}")
print(f"columns are :{df.columns}")
print(f"shape of the df before removing nan values:{df.shape}")
df = df.dropna()
print(f"shape of the df after removing nan values:{df.shape}")
print(f"datatypes of the columns in the df are:{df.dtypes}")

X = df.drop("label",axis=1)
y=df["label"]

print(f"shape of the X is:{X.shape}")
print(f"shape of the y is:{y.shape}")

messages = X.copy()
messages = messages.reset_index()

total number of rows are:20800
columns are :Index(['id', 'title', 'author', 'text', 'label'], dtype='object')
shape of the df before removing nan values:(20800, 5)
shape of the df after removing nan values:(18285, 5)
datatypes of the columns in the df are:id         int64
title     object
author    object
text      object
label      int64
dtype: object
shape of the X is:(18285, 4)
shape of the y is:(18285,)


In [22]:
corpus = []
ps = PorterStemmer()
wl = WordNetLemmatizer()
for index in range(len(messages)):
   title = re.sub("[^a-zA-Z]"," ",messages["title"][index])
   title = title.lower()
   title = title.split()

   words = [ps.stem(word) for word in title if word not in stopwords.words("english")]
  #  words = [wl.lemmatize(word) for word in title if word not in stopwords.words("english")]
   corpus.append(" ".join(words))

print(f"length of the corpus is:{len(corpus)}")
print(f"five elements in corpus are: {corpus[:5]}")

length of the corpus is:18285
five elements in corpus are: ['hous dem aid even see comey letter jason chaffetz tweet', 'flynn hillari clinton big woman campu breitbart', 'truth might get fire', 'civilian kill singl us airstrik identifi', 'iranian woman jail fiction unpublish stori woman stone death adulteri']


In [23]:

vocab_size = 10000
sent_length = 15
dim = 20
onehot = [one_hot(words,vocab_size) for words in corpus]
print(f"length of onehot representaion is:{len(onehot)}")
print(f"five elements in onehot are:{onehot[:5]}")

padded_onehot = pad_sequences(onehot,padding="pre",maxlen= sent_length)

print(f"length of the padded_onehot is:{padded_onehot}")
print(f"five elements in padded_onehot are:{padded_onehot[:5]}")




length of onehot representaion is:18285
five elements in onehot are:[[4571, 2756, 6558, 4667, 2076, 4299, 466, 5199, 2859, 3308], [8739, 3770, 1014, 8046, 3457, 2428, 4420], [6019, 9388, 7222, 7373], [3238, 7574, 4357, 6806, 1613, 6792], [9392, 3457, 6212, 3250, 6454, 6473, 3457, 1869, 612, 5859]]
length of the padded_onehot is:[[   0    0    0 ... 5199 2859 3308]
 [   0    0    0 ... 3457 2428 4420]
 [   0    0    0 ... 9388 7222 7373]
 ...
 [   0    0    0 ... 2034 7478 2543]
 [   0    0    0 ... 6176 7367 4901]
 [   0    0    0 ... 6377   80 4618]]
five elements in padded_onehot are:[[   0    0    0    0    0 4571 2756 6558 4667 2076 4299  466 5199 2859
  3308]
 [   0    0    0    0    0    0    0    0 8739 3770 1014 8046 3457 2428
  4420]
 [   0    0    0    0    0    0    0    0    0    0    0 6019 9388 7222
  7373]
 [   0    0    0    0    0    0    0    0    0 3238 7574 4357 6806 1613
  6792]
 [   0    0    0    0    0 9392 3457 6212 3250 6454 6473 3457 1869  612
  5859]]


In [29]:
model = Sequential()
model.add(Embedding(input_dim = 10000,output_dim = 20,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=["accuracy"])
print(model.summary())

X_final = np.array(padded_onehot)
y_final = np.array(y)

# X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0,random_state=42)

print(X_train.shape)
print(y_train.shape)

model.fit(X_final,y_final,epochs=15,batch_size=64)
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ (None, 15, 20)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 100)            │        48,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 745,505 (2.84 MB)

 Trainable params: 248,501 (970.71 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 497,004 (1.90 MB)

In [62]:
test_df = pd.read_csv("/kaggle/input/fake-news/test.csv")
test_df.head()
print(f"columns in the df are:{test_df.columns}")
print(f"shape of test_df is:{test_df.shape}")
test_df = test_df.fillna(" ")
print(f"shape of test_df is:{test_df.shape}")
test_messages = test_df.copy()
test_messages = test_messages.reset_index()

new_df = test_df[['id']]

test_corpus =[]
for index in range(len(test_messages)):
   title = re.sub("[^a-zA-Z]"," ",test_messages["title"][index])
   title = title.lower()
   title = title.split()
   words = [ps.stem(word) for word in title if word not in stopwords.words("english")]
  #  words = [wl.lemmatize(word) for word in title if word not in stopwords.words("english")]
   test_corpus.append(" ".join(words))
    
vocab_size = 10000
sent_length = 15
dim = 20
test_onehot = [one_hot(words,vocab_size) for words in test_corpus]
print(f"length of onehot representaion is:{len(onehot)}")
print(f"five elements in onehot are:{onehot[:5]}")

test_padded_onehot = pad_sequences(test_onehot,padding="pre",maxlen= sent_length)

X_test = np.array(test_padded_onehot)

y_pred = model.predict(X_test)

print(y_pred.shape)

columns in the df are:Index(['id', 'title', 'author', 'text'], dtype='object')
shape of test_df is:(5200, 4)
shape of test_df is:(5200, 4)
length of onehot representaion is:18285
five elements in onehot are:[[4571, 2756, 6558, 4667, 2076, 4299, 466, 5199, 2859, 3308], [8739, 3770, 1014, 8046, 3457, 2428, 4420], [6019, 9388, 7222, 7373], [3238, 7574, 4357, 6806, 1613, 6792], [9392, 3457, 6212, 3250, 6454, 6473, 3457, 1869, 612, 5859]]
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
(5200, 1)


In [63]:

y_pred = (y_pred > 0.5).astype("int32")

print(y_pred.shape)

new_df.loc[:, "label"]=y_pred

print(new_df.head())

new_df.to_csv("submission.csv",index=False)


(5200, 1)
      id  label
0  20800      0
1  20801      1
2  20802      0
3  20803      0
4  20804      1


/tmp/ipykernel_33/2218771514.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:, "label"]=y_pred
